In [34]:
%pylab
import numpy as np

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [35]:
X = np.loadtxt("data/X.csv", delimiter=",")
Y = np.loadtxt("data/Y.csv", delimiter=",")

In [36]:
P_hat = np.dot(X, Y.T)

In [37]:
np.max(P_hat[9])

0.94308451504716495

In [38]:
P_argsort = np.argsort(P_hat, axis=1)   # sort ascending and get the index that will sort P_hat

In [127]:
P_9_argsort = np.argsort(P_hat[9])

In [139]:
P_9_argsort

array([3137, 2331, 2507, ..., 1458, 1223, 1201])

In [138]:
P_9_argsort[np.size(P_9_argsort):np.size(P_9_argsort) - 4:-1]

array([1201, 1223, 1458])

In [39]:
P_hat[9][P_argsort[9]]

array([-0.33260204, -0.29911737, -0.27948714, ...,  0.83024974,
        0.87086996,  0.94308452])

In [40]:
np.where(P_hat[9] == np.max(P_hat[9]))

(array([1201]),)

In [41]:
P_hat[9][1201]

0.94308451504716495

In [44]:
P_hat[9][1201]

0.94308451504716495

In [33]:
import pandas as pd
train_data = pd.read_csv("data/train_data.csv")
train_data.shape

(14654, 6)

In [112]:
train = pd.pivot_table(train_data, values='quantity', 
                              index=['customer_id'], 
                              columns=['item_id'],
                             aggfunc=np.sum,
                             fill_value=0)   #change train from dataframe to matrix form
# row is customer, column is item and then the value is aggregated sum of quantity
train.shape

(4889, 3365)

In [48]:
R = R.values
R[9][1201]

0

In [50]:
P = np.zeros((4889, 3365))   # P is the preference matrix
P[R > 0] = 1

In [57]:
P_nonzero = np.nonzero(P)

In [75]:
P_9_nonzero = np.nonzero(P[9])
P_9_nonzero

(array([ 519,  732,  734,  879, 1233, 1241, 1404, 1413, 2038]),)

In [64]:
alpha = 40
C = np.ones((4889, 3365))     # C matrix is the confidence matrix 
C += alpha * R   # c_ui = 1 + alpha * r_ui

In [68]:
C[9][P_9_nonzero]

array([ 41.,  41.,  41.,  41.,  41.,  41.,  41.,  41.,  41.])

In [78]:
Y.T[:,P_9_nonzero[0]]

array([[ 0.58753464,  0.58159539,  1.25396378, -0.10827913,  2.37060617,
         1.47381058,  1.86182741,  0.83018451,  1.07148698],
       [-0.15111822, -0.17454834,  0.03703686,  0.18842842,  1.28020178,
         0.62326289, -0.56224869, -0.15854996, -0.14606735],
       [ 0.33568918,  0.33730624,  1.20743318,  1.46566233,  0.46635975,
         0.31148845,  2.29283223,  0.98539582,  0.29508493]])

In [80]:
I_u = np.zeros((3365, 3365))   # identity matrix
np.fill_diagonal(I_u, 1)

In [81]:
I_3 = np.zeros((3, 3))
np.fill_diagonal(I_3, 1)

In [82]:
C_u = np.zeros((3365, 3365))   # construct the diagonal matrix
for index in xrange(3365): 
    C_u[index][index] = C[9][index]
np.unique(C_u, return_counts=True)

(array([  0.,   1.,  41.]), array([11319860,     3356,        9]))

In [83]:
W_9 = inv(np.dot(Y.T, Y) + np.dot(np.dot(Y.T, C_u - I_u), Y) + 2 * I_3)

In [140]:
s_9 = np.dot(np.dot(Y[1201].T, W_9), Y.T[:, P_9_nonzero[0]])
s_9

array([ 0.0017359 ,  0.00176304,  0.00306097, -0.00060775,  0.00362022,
        0.00255981,  0.00564988,  0.00235427,  0.0029339 ])

In [145]:
s_9 = np.dot(np.dot(Y[1201], W_9), Y.T[:, P_9_nonzero[0]])
s_9

ValueError: shapes (3,) and (9,3) not aligned: 3 (dim 0) != 9 (dim 0)

In [147]:
c_9 = C[9][P_9_nonzero[0]]

In [148]:
c_9.shape

(9,)

In [103]:
p_9_1201 = np.multiply(s_9, c_9)

In [105]:
p_9_1201

array([ 0.07117179,  0.0722845 ,  0.12549983, -0.02491788,  0.14842894,
        0.10495239,  0.23164519,  0.096525  ,  0.12028979])

In [125]:
np.sum(p_9_1201)

0.94587955176863059

In [106]:
np.max(p_9_1201)

0.23164518579226917

In [164]:
np.argsort(p_9_1201)[::-1]

array([6, 4, 2, 8, 5, 7, 1, 0, 3])

In [167]:
p_9_1201[np.argsort(p_9_1201)[::-1]]

array([ 0.23164519,  0.14842894,  0.12549983,  0.12028979,  0.10495239,
        0.096525  ,  0.0722845 ,  0.07117179, -0.02491788])

In [111]:
P_9_nonzero[0][np.argsort(p_9_1201)]

array([ 879,  519,  732, 1413, 1241, 2038,  734, 1233, 1404])

In [123]:
train.iloc[9,1404]

1

In [124]:
np.max(p_9_1201) / P_hat[9][1201]

0.24562505490898054

In [150]:
train.index[9] + " " + train.columns[1404]

'000004_LAP D1793452400001'

In [165]:
index = P_9_nonzero[0][np.argsort(p_9_1201)[::-1]]
index

array([1404, 1233,  734, 2038, 1241, 1413,  732,  519,  879])